# Indianapolis Museum of Art - From Catalogue Data to Linked Art 

## Introduction

Linked Art is a community working together to create a shared Model based on Linked Open Data to describe Art. 

- This Jupyter notebook provides a documented exemplar of the transformation process, from catalogue data to Linked Art.

- Further information: https://linked.art

In [13]:
from IPython.display import IFrame
IFrame("https://linked.art", width=1000, height=350)

## Indianapolis Museum of Art (IMA) & Linked Art
The [Indianapolis Museum of Art (IMA)](https://discovernewfields.org/) has transformed a sample of its catalogue data to Linked Art JSON-LD. 

- The data are available at https://github.com/IMAmuseum/LinkedArt 
- Data were sourced from EMu in XML format (EMu source: Catalogue, Rights, Narratives, and Locations modules)
- This Jupyter notebook uses the [Objects catalogue data](https://github.com/IMAmuseum/LinkedArt/blob/master/XML/ObjectsSample.xml) to provided a documented exemplar of the transformation process, from catalogue data to Linked Art
- [raw file](https://raw.githubusercontent.com/IMAmuseum/LinkedArt/master/XML/ObjectsSample.xml) 

In [14]:
from IPython.display import IFrame
IFrame('https://discovernewfields.org/collections', width=1000, height=450)

## Catalogue Data 



## Import What We Need for Notebook

The following Python libraries are needed for the Jupyter notebook exemplar:

In [60]:
try:
    import pyjxslt
except:
    !pip install pyjxslt
    import pyjxslt
    
try:
    import lxml.etree as ET
except:
    !pip install lxml
    import lxml.etree as ET

try:
    import xmltodict
except:
    !pip install xmltodict
    import xmltodict
     
try:
    import json
except:
    !pip install json
    import json
    
try:
    import ipywidgets as widgets
except:
    !pip install ipywidgets
    import ipywidgets as widgets

In [61]:
%%html

<link rel="stylesheet" href="src/css/style.css">

<script src="https://cdn.jsdelivr.net/g/async@1.5.0,jquery@1.11.0,es6-promise@1.0.0,bootstrap@2.3.2,codemirror@3.22.0(codemirror.min.js+addon/lint/lint.js+addon/edit/matchbrackets.js+addon/edit/closebrackets.js+addon/display/placeholder.js+addon/hint/show-hint.js+mode/ntriples/ntriples.js+mode/javascript/javascript.js)"></script>
    <script src="https://cdn.jsdelivr.net/npm/jsonld@3.2.0/dist/jsonld.min.js"></script>
    <!--<script src="https://unpkg.com/jsonld@x.y.z/dist/jsonld.js"></script>-->
    <!--<script src="https://cdnjs.cloudflare.com/ajax/libs/jsonld/x.y.z/jsonld.js"></script>-->
    <script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.17/d3.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/d3-tip/0.6.7/d3-tip.min.js"></script>
    <script src="./src/js/jsonld-vis.js"></script>
    <script src="./src/js/jsonld-vis2.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/jsonld-signatures@2.3.1/dist/jsonld-signatures.min.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/node-forge@0.8.1/dist/forge.min.js"></script>
    
    
    <script src="./src/js/jsonld-vis3.js"></script>


In [62]:
%%javascript

function jsonldVis(jsonld, selector, config) {
    if (!arguments.length) return jsonldVis;
    config = config || {};

    var h = config.h || 600
      , w = config.w || 800
      , maxLabelWidth = config.maxLabelWidth || 250
      , transitionDuration = config.transitionDuration || 750
      , transitionEase = config.transitionEase || 'cubic-in-out'
      , minRadius = config.minRadius || 5
      , scalingFactor = config.scalingFactor || 2;

    var i = 0;

    var tree = d3.layout.tree()
      .size([h, w]);

    var diagonal = d3.svg.diagonal()
      .projection(function(d) { return [d.y, d.x]; });

    var svg = d3.select(selector).append('svg')
      .attr('width', w)
      .attr('height', h)
      .append('g')
      .attr('transform', 'translate(' + maxLabelWidth + ',0)');

    

    var root = jsonldTree(jsonld);
    root.x0 = h / 2;
    root.y0 = 0;
    root.children.forEach(collapse);

    function changeSVGWidth(newWidth) {
      if (w !== newWidth) {
        d3.select(selector + ' > svg').attr('width', newWidth);
      }
    }

    function jsonldTree(source) {
      var tree = {};

      if ('@id' in source) {
        tree.isIdNode = true;
        tree.name = source['@id'];
        if (tree.name.length > maxLabelWidth / 9) {
          tree.valueExtended = tree.name;
          tree.name = '...' + tree.valueExtended.slice(-Math.floor(maxLabelWidth / 9));
        }
      } else {
        tree.isIdNode = true;
        tree.isBlankNode = true;
        // random id, can replace with actual uuid generator if needed
        tree.name = '_' + Math.random().toString(10).slice(-7);
      }

      var children = [];
      Object.keys(source).forEach(function(key) {
        if (key === '@id' || key === '@context' || source[key] === null) return;

        var valueExtended, value;
        if (typeof source[key] === 'object' && !Array.isArray(source[key])) {
          children.push({
            name: key,
            children: [jsonldTree(source[key])]
          });
        } else if (Array.isArray(source[key])) {
          children.push({
            name: key,
            children: source[key].map(function(item) {
              if (typeof item === 'object') {
                return jsonldTree(item);
              } else {
                return { name: item };
              }
            })
          });
        } else {
          valueExtended = source[key];
          value = valueExtended;
          if (value.length > maxLabelWidth / 9) {
            value = value.slice(0, Math.floor(maxLabelWidth / 9)) + '...';
            children.push({
              name: key,
              value: value,
              valueExtended: valueExtended
            });
          } else {
            children.push({
              name: key,
              value: value
            });
          }
        }
      });

      if (children.length) {
        tree.children = children;
      }

      return tree;
    }

    function update(source) {
      var nodes = tree.nodes(root).reverse();
      var links = tree.links(nodes);

      nodes.forEach(function(d) { d.y = d.depth * maxLabelWidth; });

      var node = svg.selectAll('g.node')
        .data(nodes, function(d) { return d.id || (d.id = ++i); });

      var nodeEnter = node.enter()
        .append('g')
        .attr('class', 'node')
        .attr('transform', function(d) { return 'translate(' + source.y0 + ',' + source.x0 + ')'; })
        .on('click', click);

      nodeEnter.append('circle')
        .attr('r', 0)
        .style('stroke-width', function(d) {
          return d.isIdNode ? '2px' : '1px';
        })
        .style('stroke', function(d) {
          return d.isIdNode ? '#F7CA18' : '#4ECDC4';
        })
        .style('fill', function(d) {
          if (d.isIdNode) {
            return d._children ? '#F5D76E' : 'white';
          } else {
            return d._children ? '#86E2D5' : 'white';
          }
        })
        

      nodeEnter.append('text')
        .attr('x', function(d) {
          var spacing = computeRadius(d) + 5;
          return d.children || d._children ? -spacing : spacing;
        })
        .attr('dy', '4')
        .attr('text-anchor', function(d) { return d.children || d._children ? 'end' : 'start'; })
        .text(function(d) { return d.name + (d.value ? ': ' + d.value : ''); })
        .style('fill-opacity', 0);

      var maxSpan = Math.max.apply(Math, nodes.map(function(d) { return d.y + maxLabelWidth; }));
      if (maxSpan + maxLabelWidth + 20 > w) {
        changeSVGWidth(maxSpan + maxLabelWidth);
        d3.select(selector).node().scrollLeft = source.y0;
      }

      var nodeUpdate = node.transition()
        .duration(transitionDuration)
        .ease(transitionEase)
        .attr('transform', function(d) { return 'translate(' + d.y + ',' + d.x + ')'; });

      nodeUpdate.select('circle')
        .attr('r', function(d) { return computeRadius(d); })
        .style('stroke-width', function(d) {
          return d.isIdNode ? '2px' : '1px';
        })
        .style('stroke', function(d) {
          return d.isIdNode ? '#F7CA18' : '#4ECDC4';
        })
        .style('fill', function(d) {
          if (d.isIdNode) {
            return d._children ? '#F5D76E' : 'white';
          } else {
            return d._children ? '#86E2D5' : 'white';
          }
        });

      nodeUpdate.select('text').style('fill-opacity', 1);

      var nodeExit = node.exit().transition()
        .duration(transitionDuration)
        .ease(transitionEase)
        .attr('transform', function(d) { return 'translate(' + source.y + ',' + source.x + ')'; })
        .remove();

      nodeExit.select('circle').attr('r', 0);
      nodeExit.select('text').style('fill-opacity', 0);

      var link = svg.selectAll('path.link')
        .data(links, function(d) { return d.target.id; });

      link.enter().insert('path', 'g')
        .attr('class', 'link')
        .attr('d', function(d) {
          var o = { x: source.x0, y: source.y0 };
          return diagonal({ source: o, target: o });
        });

      link.transition()
        .duration(transitionDuration)
        .ease(transitionEase)
        .attr('d', diagonal);

      link.exit().transition()
        .duration(transitionDuration)
        .ease(transitionEase)
        .attr('d', function(d) {
          var o = { x: source.x, y: source.y };
          return diagonal({ source: o, target: o });
        })
        .remove();

      nodes.forEach(function(d) {
        d.x0 = d.x;
        d.y0 = d.y;
      });
    }

    function computeRadius(d) {
      if (d.children || d._children) {
        return minRadius + (numEndNodes(d) / scalingFactor);
      } else {
        return minRadius;
      }
    }

    function numEndNodes(n) {
      var num = 0;
      if (n.children) {
        n.children.forEach(function(c) {
          num += numEndNodes(c);
        });
      } else if (n._children) {
        n._children.forEach(function(c) {
          num += numEndNodes(c);
        });
      } else {
        num++;
      }
      return num;
    }

    function click(d) {
      if (d.children) {
        d._children = d.children;
        d.children = null;
      } else {
        d.children = d._children;
        d._children = null;
      }

      update(d);

      // fast-forward blank nodes
      if (d.children) {
        d.children.forEach(function(child) {
          if (child.isBlankNode && child._children) {
            click(child);
          }
        });
      }
    }

    function collapse(d) {
      if (d.children) {
        d._children = d.children;
        d._children.forEach(collapse);
        d.children = null;
      }
    }

    update(root);
  }

  if (typeof module !== 'undefined' && module.exports) {
    module.exports = jsonldVis;
  } else {
    d3.jsonldVis = jsonldVis;
  }

jsonldVis(d3);

<IPython.core.display.Javascript object>

## Data Transformation 

### Define Variables

In [63]:
baseURI      = "https://data.discovernewfields.org/"
crm          = "http://www.cidoc-crm.org/cidoc-crm/"

### Open XML File and Convert to Python dictionary

In [64]:
xml_filename = "data/ima/ObjectsSample.xml"

with open(xml_filename, 'r') as myfile:
    obj = xmltodict.parse(myfile.read())  

In [65]:
 #print(json.dumps(obj, indent=2))

In [66]:
allObjects = obj["table"]["tuple"]

#object dropdown options
objOptions = []
objOptions.append(("Please select an object",''))

for obj in allObjects:
    title = irn = ""
    # define properties variables  
    for prop in obj["atom"]:      
        propName = prop["@name"]     
        if "#text" in prop:
            if propName == "irn":
                irn = prop["#text"]  
            if propName == "TitMainTitle":
                title = prop["#text"]
            
    objOptions.append((irn + ' : ' + title,irn))
    


### Select Catalogue Item to Transform

In [67]:
selectObject = widgets.Dropdown(
        options=objOptions,
        description='Object:')

display(selectObject)

Dropdown(description='Object:', options=(('Please select an object', ''), ('1032 : long-neck vase with cup mou…

In [78]:
selectedIRN = selectObject.value
selectedObject = {}

# display selected record 
for obj in allObjects:
    for prop in obj["atom"]:         
        if prop["@name"] == "irn":
            irn = prop["#text"]
            if irn == selectedIRN:
                selectedObject = obj
               
print(json.dumps(selectedObject,indent=2))

{
  "atom": [
    {
      "@name": "irn",
      "@type": "text",
      "@size": "short",
      "#text": "51330"
    },
    {
      "@name": "AdmPublishWebNoPassword",
      "@type": "text",
      "@size": "short",
      "#text": "Yes"
    },
    {
      "@name": "TitAccessionNo",
      "@type": "text",
      "@size": "short",
      "#text": "30.388"
    },
    {
      "@name": "TitPreviousAccessionNo",
      "@type": "text",
      "@size": "short",
      "#text": "TR1548/301"
    },
    {
      "@name": "TitObjectStatus",
      "@type": "text",
      "@size": "short",
      "#text": "Accessioned"
    },
    {
      "@name": "TitAccessionDate",
      "@type": "text",
      "@size": "short",
      "#text": "1930-10-21"
    },
    {
      "@name": "TitMainTitle",
      "@type": "text",
      "@size": "short",
      "#text": "Towing Boat at Yotsugi D\u014dri [40] (Yotsugi d\u014dri no hikifune)"
    },
    {
      "@name": "TitSeriesTitle",
      "@type": "text",
      "@size": "short",
  

### Minimum Linked Art representation

<a id='core_properties'></a>

#### Core Properties
https://linked.art/model/base/#core-properties

There are a few core properties that every resource should have for it to be a useful part of the world of Linked Open Data:

- @context
- id
- type
- _label
The simplest possible object has a URI, a class and a label.

##### Mapping
- The `id` is a URL and has been created from the `irn` value together with a URL prefix: https://data.discovernewfields.org/

- The `_label` is a human readable label, intended for developers and other people reading the data. The value is taken from the `TitMainTitle` property.

In [80]:
# create dictionary to hold object desription
minla = {}
obj = selectedObject

# define properties variables  
for prop in obj["atom"]:      
    propName = prop["@name"]     
    if "#text" in prop:
        if propName == "irn":
            irn = prop["#text"]  
        if propName == "TitMainTitle":
            title = prop["#text"]
"""            
for table in obj["table"]:
    if table["@name"] == "Homepage":
        print(table)
        for atom in obj:
            if atom["@name"] == "EleIdentifier":
                homepageID = atom["#text"]
 

<table name="Homepage">
      <tuple>
        <atom name="RefBibliographicNotes">80085</atom>
        <atom name="irn">23278</atom>
        <atom name="EleTitle">Dagwood ID for 06.137</atom>
        <atom name="EleIdentifier">80085</atom>
      </tuple>

"""

# minimum Linked Art properties
minla["@context"] = "https://linked.art/ns/v1/linked-art.json"
minla["id"] = baseURI + "object/" + irn
minla["type"] = "HumanMadeObject"
minla["_label"] = title 

min = {}
min.update(minla)

f = open("./data/ima/linkedart/min.json", "w")
f.write(json.dumps(min, indent=2))
f.close() 


minla

{'@context': 'https://linked.art/ns/v1/linked-art.json',
 'id': 'https://data.discovernewfields.org/object/51330',
 'type': 'HumanMadeObject',
 '_label': 'Towing Boat at Yotsugi Dōri [40] (Yotsugi dōri no hikifune)'}

In [71]:
%%html
<script>
d3.json("./data/ima/linkedart/min.json", (err, data) => {
  if (err) return console.warn(err);
  d3.jsonldVis(data, '#min', { w: 800, h: 200, maxLabelWidth: 200 });
});
</script>
<div id="min" class="svg-small"/>

### Identifiers

https://linked.art/model/base/#identifiers

Many resources of interest are also given external identifiers, such as accession numbers for objects, ORCIDs for people or groups, lot numbers for auctions, and so forth. Identifiers are represented in a very similar way to names, but instead use the Identifier class. Identifiers will normally have a classification determining which sort of identifier it is, to distinguish between internal repository system assigned numbers from museum assigned accession numbers, for example.

As Identifiers and Names use the same `identified_by` property, the JSON will frequently have mixed classes in the array. Unlike Names, Identifiers are not part of human language and thus cannot have translations or a language associated with them.

In [81]:
jdoc = None
jdoc = {}

# define properties variables  
for prop in obj["atom"]:      
    propName = prop["@name"]        
    if "#text" in prop:
        if propName == "irn":
            irn = prop["#text"]    
        if propName == "TitAccessionNo":
            titAccessionNo = prop["#text"]
                               
jdoc["identified_by"] = []          
jdoc["identified_by"].append({
        "id": baseURI + "object/" + irn + "/irn",
        "type": "Identifier",
        "_label": "IMA at Newfields Collections Database Number for the Object",
        "content": irn,
        "classified_as": [{
            "id": "http://vocab.getty.edu/aat/300404621",
            "type": "Type",
            "_label": "repository numbers"
                        }]
                })
                   
jdoc["identified_by"].append({
        "id": baseURI + "/>object/" + irn + "/object-number",
        "type": "Identifier",
        "_label": "IMA at Newfields Object Number for the Object",
        "content": titAccessionNo,
        "classified_as": [{
            "id": "http://vocab.getty.edu/aat/300312355",
            "type": "Type",
            "_label": "accession numbers"
                        }]
                    })

identifiers = jdoc
frag = {}
frag.update(minla) 
frag.update(identifiers)
print(json.dumps(frag, indent=2))   

f = open("./data/ima/linkedart/ids.json", "w")
f.write(json.dumps(frag, indent=2))
f.close() 

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/object/51330",
  "type": "HumanMadeObject",
  "_label": "Towing Boat at Yotsugi D\u014dri [40] (Yotsugi d\u014dri no hikifune)",
  "identified_by": [
    {
      "id": "https://data.discovernewfields.org/object/51330/irn",
      "type": "Identifier",
      "_label": "IMA at Newfields Collections Database Number for the Object",
      "content": "51330",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300404621",
          "type": "Type",
          "_label": "repository numbers"
        }
      ]
    },
    {
      "id": "https://data.discovernewfields.org//>object/51330/object-number",
      "type": "Identifier",
      "_label": "IMA at Newfields Object Number for the Object",
      "content": "30.388",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300312355",
          "type": "Type",
          "_label": "accession numb

In [82]:
%%html
<script>
d3.json("./data/ima/linkedart/ids.json", (err, data) => {
  if (err) return console.warn(err);
  d3.jsonldVis(data, '#ids', { w: 1000, h: 300, maxLabelWidth: 250 });
});
</script>
<div id="ids" class="svg-small"/>

### Names

https://linked.art/model/base/#names

As the `_label` property is intended as internal documentation for the data, it is strongly recommended that every resource that should be rendered to an end user also have at least one specific name. The name could be for an object, a person, a group, an event or anything else. This pattern uses the `identified_by` property, with a `Name` resource. The value of the name is given in the content property of the `Name`.

It is somewhat unintuitive to think of a name as identifying the resource it is associated with, as names are typically not unique. However, as the name itself is uniquely identified rather than just an anonymous string, they are no longer a shared label and instead the particular instance of a name is uniquely associated with the resource. With this formulation, the name instance does uniquely identify the resource.

If there is more than one name given, then there should be one that is `classified_as` the primary name for use. This is done by adding the `Primary Name (aat:300404670) term to it. There should be exactly one primary title given per language.

Names are also part of human communication, and can have the Linguistic features of the model associated with them, such as having a particular language, or having translations.

In [83]:
jdoc = {}
jdoc["identified_by"] = []

# define properties variables  
for prop in obj["atom"]:      
    propName = prop["@name"] 
    if "#text" in prop:
        if propName == "irn":
            irn = prop["#text"]   
        if propName == "TitMainTitle":
            title = prop["#text"]
            

jdoc["identified_by"].append({
        "id": baseURI + "object/" + irn + "/title",
        "type": "Name",
        "_label": "Primary Title for the Object",
        "content": title ,
        "classified_as": [{
        "id": "http://vocab.getty.edu/aat/300404670",
        "type": "Type",
        "_label": "preferred terms"
                        }]
                })
try:   
    if obj["table"]["@name"] == "AltTitles":
        x = 0
        for tuple in obj["table"]["tuple"]:
            x +=1
            for atom in tuple:
                if atom["@name"] == "TitAlternateTitles":
                    content = atom["#text"]
                else:
                    content = ""
                    
                jdoc["identified_by"].append({
                    "id": baseURI + "object/" + irn + "/alt-title-" + x,
                    "type": "Name",
                    "_label": "Alternate Title for the Object",
                    "content": content,
                    "classified_as": [{
                            "id": "http://vocab.getty.edu/aat/300417227",
                            "type": "Type",
                            "_label": "alternate titles"}]   
                    })
except:
    pass
       
titles = jdoc   
frag = {}
frag.update(minla)
frag.update(titles)
print(json.dumps(frag, indent=2))

f = open("./data/ima/linkedart/name.json", "w")
f.write(json.dumps(frag, indent=2))
f.close() 

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/object/51330",
  "type": "HumanMadeObject",
  "_label": "Towing Boat at Yotsugi D\u014dri [40] (Yotsugi d\u014dri no hikifune)",
  "identified_by": [
    {
      "id": "https://data.discovernewfields.org/object/51330/title",
      "type": "Name",
      "_label": "Primary Title for the Object",
      "content": "Towing Boat at Yotsugi D\u014dri [40] (Yotsugi d\u014dri no hikifune)",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300404670",
          "type": "Type",
          "_label": "preferred terms"
        }
      ]
    }
  ]
}


In [86]:
%%html
<script>
d3.json("./data/ima/linkedart/name.json", (err, data) => {
  if (err) return console.warn(err);
  d3.jsonldVis(data, '#name', { w: 1000, h: 300, maxLabelWidth: 250 });
});
</script>
<div id="name" class="svg-small"/>

### Classification

https://linked.art/model/base/#types-and-classifications

CIDOC-CRM is a framework that must be extended via additional vocabularies and ontologies to be useful. The provided mechanism for doing this is the classified_as property, which refers to a term from a controlled vocabulary. This is in contrast to the `type` property, which is used for CIDOC-CRM defined classes, and a few extensions as needed. 

The `classified_as` property is thus a way to be more specific about the sort of entity, while maintaining the core information as the class using type. Controlled vocabulary entries should not be used with `type`, nor classes used with `classified_as`.

While any external vocabulary of terms can be used, the Getty's Art and Architecture Thesaurus is used whenever possible for consistency and that it is already widespread in the museum domain. The set of terms that have been identified as useful are listed in the community best-practices for recommendations, and within the documentation of the model when a particular choice is essential for interoperability.

In [87]:
jdoc = {}

# define properties variables  
for prop in obj["atom"]:      
    propName = prop["@name"] 
        
    if "#text" in prop:
        if propName == "TitObjectType":
            titleObjectType = prop["#text"]
        if propName == "irn":
            irn = prop["#text"]  
                           
phyMediaCategory = ""          
for table in obj["table"]:
    tableName = table["@name"]
    if tableName == "ObjectTypes":
        try:
            for atom in table["tuple"]:
                if atom["atom"]["@name"] == "PhyMediaCategory":
                    phyMediaCategory = atom["atom"]["#text"]
        except:
            pass
            
if titleObjectType.startswith("Visual Work"):
    jdoc["classified_as"] = [{
                    "id": "http://vocab.getty.edu/aat/300133025",
                    "type": "Type",
                    "_label": "works of art" 
                }]
                
if "Drawings" in titleObjectType:
    jdoc["classified_as"] = [{
                    "id": "http://vocab.getty.edu/aat/300033973",
                    "type": "Type",
                    "_label": "drawings (visual works)" 
                }]
                
if "Multimedia" in titleObjectType:
    jdoc["classified_as"] = [{
                    "id": "http://vocab.getty.edu/aat/300047910",
                    "type": "Type",
                    "_label": "multimedia works" 
                }]
                
if "Needlework" in titleObjectType:
    jdoc["classified_as"] = [{
                    "id": "http://vocab.getty.edu/aat/300264072",
                    "type": "Type",
                    "_label": "needlework (visual works)" 
                }]
                
if "Paintings" in titleObjectType:
    jdoc["classified_as"] = [{
                    "id": "http://vocab.getty.edu/aat/300033618",
                    "type": "Type",
                    "_label": "paintings (visual works)" 
                }] 
                
if "Pastel" in titleObjectType:
    jdoc["classified_as"] = [{
                    "id": "http://vocab.getty.edu/aat/300076922",
                    "type": "Type",
                    "_label": "pastels (visual works)" 
                }] 
if "Performance" in titleObjectType:
    jdoc["classified_as"] = [{
                    "id": "http://vocab.getty.edu/aat/300121445",
                    "type": "Type",
                    "_label": "performance art" 
                }] 
if "Photograph" in titleObjectType:
    jdoc["classified_as"] = [{
                    "id": "http://vocab.getty.edu/aat/300046300",
                    "type": "Type",
                    "_label": "photographs" 
                }]
if "Prints" in titleObjectType:
    jdoc["classified_as"] = [{
                    "id": "http://vocab.getty.edu/aat/300041273",
                    "type": "Type",
                    "_label": "prints (visual works)" 
                }]
if "Sculpture" in titleObjectType:
    jdoc["classified_as"] = [{
                    "id": "http://vocab.getty.edu/aat/300047090",
                    "type": "Type",
                    "_label": "sculpture (visual works)" 
                }]
                
if phyMediaCategory != "":  
    try:
        atoms = table["tuple"]["atom"]
        for atom in atoms:
            if atom["@name"] == "PhyMediaCategory":
                    value = atom["#text"]
                    c = {
                    "id": baseURI + "thesauri/type/",
                    "type": "Type",
                    "_label": value
                    }
                    jdoc["classified_as"].append(c)
    except:
        pass
 
classification = jdoc
frag = {}
frag.update(minla)
frag.update(classification)
print(json.dumps(frag, indent=2))

f = open("./data/ima/linkedart/class.json", "w")
f.write(json.dumps(frag, indent=2))
f.close() 

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/object/51330",
  "type": "HumanMadeObject",
  "_label": "Towing Boat at Yotsugi D\u014dri [40] (Yotsugi d\u014dri no hikifune)",
  "classified_as": [
    {
      "id": "http://vocab.getty.edu/aat/300041273",
      "type": "Type",
      "_label": "prints (visual works)"
    }
  ]
}


In [91]:
%%html
<script>
d3.json("./data/ima/linkedart/class.json", (err, data) => {
  if (err) return console.warn(err);
  d3.jsonldVis(data, '#class', { w: 1000, h: 300, maxLabelWidth: 250 });
});
</script>
<div id="class" class="svg-small"/>

### Materials

https://linked.art/model/object/physical/#materials

Objects are created using different materials, such as canvas or marble. These are recorded using the `made_of` property on the object directly. The materials are the type of material, rather than the specific bits of matter and therefore refer to entries in external vocabularies. When possible, it is good to use this model, and combined with the parts model described in the next section, allows for a comprehensive set of information about which parts are which sizes, shapes, colors, and made of which materials.

Note that the type-of-type pattern is not needed for materials, like it is for shape, as they have their own `Material` class that is used to distinguish them.



In [92]:
jdoc = {}

made_of = []

# # <xsl:value-of select="lower-case(translate(replace(.,'[^a-zA-Z0-9 ]',''), ' ', '-'))"/>"
'''
for table in obj["table"]:
    if table["@name"] == "Medium":
        atom = table["tuple"].get("atom")    
        if atom.get("@name") == 'PhyMedium':
            value = atom.get("#text")
            made_of.append({
                    "id": baseURI + "thesauri/material/" + value,
                    "type": "Material",
                    "_label": "Material of Which the Object is Composed",
                    "content": value    
                    })
    if table["@name"] == "Support":
        atom = table["tuple"].get("atom")
        if atom.get("@name") == 'PhySupport':
            value = atom.get("#text")
            made_of.append({
                    "id": baseURI + "thesauri/material/" + value,
                    "type": "Material",
                    "_label": "Material of Which the Object is Composed",
                    "content": value    
                    })
'''
if len(made_of) > 0:
    jdoc["made_of"] = made_of

materials = {}
materials.update(jdoc)
    
frag = {}
frag.update(minla)
frag.update(materials)
print(json.dumps(frag, indent=2))
    


{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/object/51330",
  "type": "HumanMadeObject",
  "_label": "Towing Boat at Yotsugi D\u014dri [40] (Yotsugi d\u014dri no hikifune)"
}


## Write file

Bring the separate descriptions together and write to a JSON-LD file

In [93]:
# write linked art to file

jdoc = {}

jdoc.update(minla)
if materials:
    jdoc.update(materials)
jdoc.update(identifiers)
jdoc.update(classification)
jdoc["identified_by"][0].update(titles["identified_by"][0])

print(json.dumps(jdoc, indent=2)) 

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/object/51330",
  "type": "HumanMadeObject",
  "_label": "Towing Boat at Yotsugi D\u014dri [40] (Yotsugi d\u014dri no hikifune)",
  "identified_by": [
    {
      "id": "https://data.discovernewfields.org/object/51330/title",
      "type": "Name",
      "_label": "Primary Title for the Object",
      "content": "Towing Boat at Yotsugi D\u014dri [40] (Yotsugi d\u014dri no hikifune)",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300404670",
          "type": "Type",
          "_label": "preferred terms"
        }
      ]
    },
    {
      "id": "https://data.discovernewfields.org//>object/51330/object-number",
      "type": "Identifier",
      "_label": "IMA at Newfields Object Number for the Object",
      "content": "30.388",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300312355",
          "type": "Type",
         

In [94]:
filepath = "./data/ima/linkedart/" + irn + ".json"

f = open(filepath, "w")
f.write(json.dumps(jdoc, indent=2))
f.close() 

f = open("./data/ima/linkedart/vis.json", "w")
f.write(json.dumps(jdoc, indent=2))
f.close() 

## Linked Art - Data Visualisation

In [96]:
%%html


<script>
d3.json("./data/ima/linkedart/vis.json", (err, data) => {
  if (err) return console.warn(err);
  d3.jsonldVis(data, '#visualized', { w: 800, h: 600, maxLabelWidth: 250 });
});
</script>
<div id="visualized" class="svg" ></div>